# Data generation for Trappist 1-e models from Bower et al. (2024)

In [ ]:
import copy
import logging

import numpy as np
import pandas as pd
from pathlib import Path

from atmodeller import debug_logger
from atmodeller.constraints import ElementMassConstraint, SystemConstraints, BufferedFugacityConstraint
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer
from atmodeller.core import GasSpecies, Species, LiquidSpecies, SolidSpecies
from atmodeller.interior_atmosphere import Planet, InteriorAtmosphereSystem
from atmodeller.initial_solution import InitialSolutionRegressor, InitialSolutionSwitchRegressor, InitialSolutionDict, InitialSolutionLast
from atmodeller.utilities import earth_oceans_to_kg, bulk_silicate_earth_abundances
from atmodeller.output import Output

logger = debug_logger()
# logger.setLevel(logging.INFO)

Parameters for the simulations.

In [ ]:
number_of_realisations = 10000
training_steps = 250

In [ ]:
surface_temperature = 1800

Species to consider. These are initially excluding solubility.

In [ ]:
H2O_g = GasSpecies("H2O")
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2")
CH4_g = GasSpecies("CH4")
N2_g = GasSpecies("N2")
NH3_g = GasSpecies("NH3")
S2_g = GasSpecies("S2")
H2S_g = GasSpecies("H2S")
SO2_g = GasSpecies("SO2")
SO_g = GasSpecies("SO")
Cl2_g = GasSpecies("Cl2")

species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, H2S_g, SO2_g, SO_g, Cl2_g])

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [ ]:
mantle_mass = 2.912E24
planet_mass = mantle_mass / (1-0.295334691460966)
trappist1e = Planet(surface_temperature=surface_temperature, planet_mass=2.912E24, surface_radius=5.861E6)

Earth planet properties, which are required to scale the bulk volatile inventories for Trappist-1e. Default parameters are Earth so we only need to specify the temperature.

In [ ]:
earth = Planet(surface_temperature=surface_temperature)

In [ ]:
earth_bse = bulk_silicate_earth_abundances()

Compute the reservoir sizes for TRAPPIST 1-e, assuming the same ppmw as Earth:

In [ ]:
trappist1e_bse = copy.deepcopy(earth_bse)
mass_scale_factor = trappist1e.mantle_mass / earth.mantle_mass

for element, values in trappist1e_bse.items():
    trappist1e_bse[element] = {key: value*mass_scale_factor for key, value in values.items()}

trappist1e_bse

Now set up the main driver of the Monte Carlo (MC) approach. This establishes the ranges over which we sample certain properties.

In [ ]:
def monte_carlo(interior_atmosphere: InteriorAtmosphereSystem, bse: dict, 
    number_of_realisations:int=100):
    """Monte Carlo driver
    
    Args:
        interior_atmosphere: An interior-atmosphere system
        bse: Dictionary of element masses, fixed to the ppmw of the bulk silicate Earth
        number_of_realisation: Number of simulations to perform
    """
    # Parameters are normally distributed between bounds.
    log10_number_ocean_moles = np.random.uniform(-1, 1, number_of_realisations) # log10 units
    log10_ch_ratios = np.random.uniform(-1, 1, number_of_realisations) # log10 units
    fo2_shifts = np.random.uniform(-5, 5, number_of_realisations) # log10 units

    for realisation in range(number_of_realisations):
        number_ocean_moles = 10**log10_number_ocean_moles[realisation]
        mass_H = earth_oceans_to_kg(number_ocean_moles)
        CH_ratio = 10**log10_ch_ratios[realisation]
        mass_C = mass_H * CH_ratio
        constraints = SystemConstraints([
            ElementMassConstraint("H", mass_H),
            ElementMassConstraint("C", mass_C),
            ElementMassConstraint("N", bse['N']['mean']),
            ElementMassConstraint("S", bse['S']['mean']),
            ElementMassConstraint("Cl", bse['Cl']['mean']),
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=fo2_shifts[realisation]))
        ])

        # Extra quantities to write to the output
        # For example, it's often helpful to have the constraints expressed in a more convenient
        # form for analysis and plotting.
        extra = {'fO2 (delta IW)': fo2_shifts[realisation], 'C/H ratio': CH_ratio,
            'Number of ocean moles': number_ocean_moles}

        interior_atmosphere.solve(constraints, extra_output=extra, tol=1e-5)

### CHONSCl with no solubilities

Start with a dictionary of initial guesses and then switch to a regressor to provide a training dataset.

In [ ]:
values = {H2_g: 100.0, H2O_g: 100.0, CO2_g: 100.0, CO_g: 500.0, S2_g: 1}
initial_solution_start = InitialSolutionDict(values, species=species)
initial_solution_training = InitialSolutionSwitchRegressor(initial_solution_start, species=species, fit=True, fit_batch_size=100, partial_fit=False)

Now run this training model and save the output to a pickle file.

In [ ]:
trappist1e_no_sols_training_path = Path(f"trappist1e_{surface_temperature}K_no_sols_{training_steps}its")

In [ ]:
trappist1e_no_sols_training = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_training, planet=trappist1e)
monte_carlo(trappist1e_no_sols_training, trappist1e_bse, number_of_realisations=training_steps)
trappist1e_no_sols_training.output(file_prefix=trappist1e_no_sols_training_path, to_pickle=True, to_excel=True)

Now, we can use the training output generated above to train a network and use this to improve convergence and speed up the computation of the final run.

In [ ]:
initial_solution_full = InitialSolutionRegressor.from_pickle(trappist1e_no_sols_training_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=500)

Run the full model and export the data.

In [ ]:
trappist1e_no_sols_path = Path(f"trappist1e_{surface_temperature}K_no_sols_{number_of_realisations}its")

In [ ]:
trappist1e_no_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_full, planet=trappist1e)
monte_carlo(trappist1e_no_sols, trappist1e_bse, number_of_realisations=number_of_realisations)
trappist1e_no_sols.output(file_prefix=trappist1e_no_sols_path, to_pickle=True, to_excel=True)

### CHONSCl with solubilities

This will set all the solubilities (where possible) to conform to a basaltic composition.

In [ ]:
trappist1e.melt_composition = 'basalt'

The pickle file output for the training model without solubilities (above) is used to inform the initial condition for the training set with solubilities, but because solubilities can strongly affect the solution, we use the `species_fill` dictionary to modify some initial guesses.

In [ ]:
species_fill = {Cl2_g: 1E-6, H2_g: 1, H2O_g: 1, S2_g: 1E-6, SO2_g: 1E-6, SO_g: 1E-6}
initial_solution_training = InitialSolutionRegressor.from_pickle(trappist1e_no_sols_training_path.with_suffix(".pkl"), species=species, species_fill=species_fill, fit=True, fit_batch_size=100, partial_fit=False)

Run the training model and save the output to a pickle file.

In [ ]:
trappist1e_with_sols_training_path = Path(f"trappist1e_{surface_temperature}K_with_sols_{training_steps}its")

In [ ]:
trappist1e_with_sols_training = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_training, planet=trappist1e)
monte_carlo(trappist1e_with_sols_training, trappist1e_bse, number_of_realisations=training_steps)
trappist1e_with_sols_training.output(file_prefix=trappist1e_with_sols_training_path, to_pickle=True, to_excel=True)

Now, we can use the training output generated above to train a network and use this to improve convergence and speed up the computation of the final run.

In [ ]:
initial_solution_full = InitialSolutionRegressor.from_pickle(trappist1e_with_sols_training_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=500)

Run the full model and export the data.

In [ ]:
trappist1e_with_sols_path = Path(f"trappist1e_{surface_temperature}K_with_sols_{number_of_realisations}its")

In [ ]:
trappist1e_with_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_full, planet=trappist1e)
monte_carlo(trappist1e_with_sols, trappist1e_bse, number_of_realisations=number_of_realisations)
trappist1e_with_sols.output(file_prefix=trappist1e_with_sols_path, to_pickle=True, to_excel=True)

### CHONSCl cooled/condensed atmospheres

Finally, read back in the high temperature atmospheres and cool them to the equilibrium temperature of Trappist-1e

In [ ]:
equilibrium_temperature = 280

This duplicates the planetary properties set above, but sets a new surface temperature.

In [ ]:
mantle_mass = 2.912E24
planet_mass = mantle_mass / (1-0.295334691460966)
trappist1e = Planet(surface_temperature=equilibrium_temperature, planet_mass=2.912E24, surface_radius=5.861E6, mantle_melt_fraction=0)

Species to consider, including condensed C and H2O

In [ ]:
H2O_g = GasSpecies("H2O")
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2")
CH4_g = GasSpecies("CH4")
#N2_g = GasSpecies("N2")
#NH3_g = GasSpecies("NH3")
#S2_g = GasSpecies("S2")
#H2S_g = GasSpecies("H2S")
#SO2_g = GasSpecies("SO2")
#SO_g = GasSpecies("SO")
#Cl2_g = GasSpecies("Cl2")
H2O_l = LiquidSpecies("H2O")
C_cr = SolidSpecies("C")

#species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, H2S_g, SO2_g, SO_g, Cl2_g, H2O_l, C_cr])
species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, H2O_l, C_cr])

system = InteriorAtmosphereSystem(species=species, planet=trappist1e)

Get element abundances for constraints

In [ ]:
trappist1e_with_sols_path = Path(f"trappist1e_{surface_temperature}K_with_sols_{number_of_realisations}its")
output = Output.read_pickle(trappist1e_with_sols_path.with_suffix('.pkl'))
with_solubility_data = output(to_dataframes=True)

In [ ]:
# Create a dictionary with the series
data = {
    'H_total': with_solubility_data["H_total"]["atmosphere_mass"],
    #'S_total': with_solubility_data["S_total"]["atmosphere_mass"],
    #'N_total': with_solubility_data["N_total"]["atmosphere_mass"],
    'O_total': with_solubility_data["O_total"]["atmosphere_mass"],
    'C_total': with_solubility_data["C_total"]["atmosphere_mass"],
    #'Cl_total': with_solubility_data["Cl_total"]["atmosphere_mass"]
}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data)

In [ ]:
ic_first = InitialSolutionDict({CH4_g: 0.9185445957233848,
    CO2_g: 5.56633357640032e-07,
    CO_g: 2.4059392096609617e-15,
    C_cr: 1.0,
    H2O_g: 0.009803396976251115,
    H2O_l: 1.0,
    H2_g: 1.2993269399660158e-05,
    O2_g: 1.5251879583110059e-80},
    species=species)

initial_solution = InitialSolutionLast(ic_first, species=species)

In [ ]:
trappist1e_cooled_path = Path(f"trappist1e_{equilibrium_temperature}K_{number_of_realisations}its")
initial_solution = InitialSolutionRegressor.from_pickle(trappist1e_cooled_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=50)

In [ ]:
for row in df.itertuples(index=True):
    constraints = SystemConstraints([
        ElementMassConstraint("H", row.H_total),
        #ElementMassConstraint("S", row.S_total),
        #ElementMassConstraint("N", row.N_total),
        ElementMassConstraint("O", row.O_total),
        ElementMassConstraint("C", row.C_total),
        #ElementMassConstraint("Cl", row.Cl_total),
        ]
    )
    system.solve(constraints, initial_solution=initial_solution)
    if row.Index == 101:
        break

In [ ]:
system.failed_solves

In [ ]:
trappist1e_cooled_path = Path(f"trappist1e_{equilibrium_temperature}K_{number_of_realisations}its")
system.output(file_prefix=trappist1e_cooled_path, to_excel=True, to_pickle=True)